In [292]:
import pandas as pd
import glob
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import hex_to_rgb
from nhl_themes import teamColours
pd.set_option('mode.chained_assignment', None)

In [293]:
data = pd.read_csv("EH_std_gl_stats_all_playoffs_adj_2024-05-13.csv")
# data = data.loc[data["xGA"] > data["xGA"].quantile(0.25)]
data = data.loc[data['GP'] > 12]
data = data.loc[data["xGA"] < 200]
data["Player"] = data["Player"].str.split(n=1).str[1]


In [294]:
data

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx
1,Hill,22-24,VGK,G,19,1089.61,37.67,545.99,783.10,54.27,93.10,95.19,93.07,2.12,14.85,16.60
7,Georgiev,22-24,COL,G,15,899.70,41.65,429.75,628.54,40.65,90.31,93.37,93.53,-0.16,0.07,-1.00
12,Khudobin,18-20,DAL,G,25,1440.21,66.45,803.51,1085.03,76.72,91.73,93.88,92.93,0.95,1.02,10.27
13,Niemi,09-10,CHI,G,22,1321.85,58.18,640.42,894.93,59.24,90.92,93.50,93.38,0.12,0.82,1.06
14,Niemi,10-14,S.J,G,40,2340.00,108.72,1145.15,1612.58,92.93,90.51,93.26,94.24,-0.98,-14.29,-15.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Bobrovsky,13-19,CBJ,G,27,1725.25,87.71,959.49,1283.73,76.31,90.86,93.17,94.06,-0.89,-6.01,-11.40
166,Bobrovsky,19-24,FLA,G,45,2650.88,125.95,1348.89,1954.21,143.12,90.66,93.55,92.68,0.88,-1.45,17.17
170,Skinner,22-24,EDM,G,20,1084.12,62.47,507.53,700.13,49.61,87.69,91.08,92.91,-1.84,-13.34,-12.86
172,Greiss,15-20,NYI,G,16,947.70,37.65,486.98,682.97,38.92,92.27,94.49,94.30,0.19,2.91,1.27


In [295]:
data["GSAx/60"] = round((60 * data["GSAx"]) / data["TOI"], 2)
data.head()

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60
1,Hill,22-24,VGK,G,19,1089.61,37.67,545.99,783.10,54.27,93.10,95.19,93.07,2.12,14.85,16.60,0.91
7,Georgiev,22-24,COL,G,15,899.70,41.65,429.75,628.54,40.65,90.31,93.37,93.53,-0.16,0.07,-1.00,-0.07
12,Khudobin,18-20,DAL,G,25,1440.21,66.45,803.51,1085.03,76.72,91.73,93.88,92.93,0.95,1.02,10.27,0.43
13,Niemi,09-10,CHI,G,22,1321.85,58.18,640.42,894.93,59.24,90.92,93.50,93.38,0.12,0.82,1.06,0.05
14,Niemi,10-14,S.J,G,40,2340.00,108.72,1145.15,1612.58,92.93,90.51,93.26,94.24,-0.98,-14.29,-15.79,-0.40


In [296]:
data["GA/100 xGA"] = round((100 * data["GA"] / data["xGA"]))
data.head()

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60,GA/100 xGA
1,Hill,22-24,VGK,G,19,1089.61,37.67,545.99,783.10,54.27,93.10,95.19,93.07,2.12,14.85,16.60,0.91,69.0
7,Georgiev,22-24,COL,G,15,899.70,41.65,429.75,628.54,40.65,90.31,93.37,93.53,-0.16,0.07,-1.00,-0.07,102.0
12,Khudobin,18-20,DAL,G,25,1440.21,66.45,803.51,1085.03,76.72,91.73,93.88,92.93,0.95,1.02,10.27,0.43,87.0
13,Niemi,09-10,CHI,G,22,1321.85,58.18,640.42,894.93,59.24,90.92,93.50,93.38,0.12,0.82,1.06,0.05,98.0
14,Niemi,10-14,S.J,G,40,2340.00,108.72,1145.15,1612.58,92.93,90.51,93.26,94.24,-0.98,-14.29,-15.79,-0.40,117.0


In [297]:
i = 0
dataCopy = data.copy()

for row in dataCopy['Player']:
    if dataCopy['Player'].value_counts()[row] > 1:
        data.iloc[i, 0] = f'{data.iloc[i, 0]} ({data.iloc[i, 2]})'
    i += 1
        
data

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60,GA/100 xGA
1,Hill,22-24,VGK,G,19,1089.61,37.67,545.99,783.10,54.27,93.10,95.19,93.07,2.12,14.85,16.60,0.91,69.0
7,Georgiev,22-24,COL,G,15,899.70,41.65,429.75,628.54,40.65,90.31,93.37,93.53,-0.16,0.07,-1.00,-0.07,102.0
12,Khudobin,18-20,DAL,G,25,1440.21,66.45,803.51,1085.03,76.72,91.73,93.88,92.93,0.95,1.02,10.27,0.43,87.0
13,Niemi (CHI),09-10,CHI,G,22,1321.85,58.18,640.42,894.93,59.24,90.92,93.50,93.38,0.12,0.82,1.06,0.05,98.0
14,Niemi (S.J),10-14,S.J,G,40,2340.00,108.72,1145.15,1612.58,92.93,90.51,93.26,94.24,-0.98,-14.29,-15.79,-0.40,117.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Bobrovsky (CBJ),13-19,CBJ,G,27,1725.25,87.71,959.49,1283.73,76.31,90.86,93.17,94.06,-0.89,-6.01,-11.40,-0.40,115.0
166,Bobrovsky (FLA),19-24,FLA,G,45,2650.88,125.95,1348.89,1954.21,143.12,90.66,93.55,92.68,0.88,-1.45,17.17,0.39,88.0
170,Skinner,22-24,EDM,G,20,1084.12,62.47,507.53,700.13,49.61,87.69,91.08,92.91,-1.84,-13.34,-12.86,-0.71,126.0
172,Greiss,15-20,NYI,G,16,947.70,37.65,486.98,682.97,38.92,92.27,94.49,94.30,0.19,2.91,1.27,0.08,97.0


In [298]:
for row in data.index:
    print(data.loc[row, 'Player'])

Hill
Georgiev
Khudobin
Niemi (CHI)
Niemi (S.J)
Raanta
Bishop (T.B)
Bishop (DAL)
Boucher
Elliott
Talbot
Ward
Hart
Osgood
Hellebuyck
Anderson
Kuemper
Dubnyk
Roloson
Nabokov
Andersen (ANA)
Andersen (TOR)
Andersen (CAR)
Shesterkin
Samsonov
Sorokin
Campbell
Markstrom
Allen
Oettinger
Halak
Swayman
Howard
Gibson
Hiller
Binnington
Saros
Lehtonen
Fleury
Biron
Brodeur
Jones
Turco
Murray
Leighton
Kiprusoff
Smith (ARI)
Smith (EDM)
Khabibulin
Francouz
Mrazek
Grubauer (COL)
Grubauer (SEA)
Luongo
Lehner
Miller
Varlamov (WSH)
Varlamov (NYI)
Bobrovsky (CBJ)
Bobrovsky (FLA)
Skinner
Greiss
Thomas


In [299]:
fig = px.scatter(data, x='GP', y='GSAx')
for row in data.index:
    fig.add_annotation(x=data.loc[row, 'GP'], y=data.loc[row, 'GSAx'], 
            text=data.loc[row, 'Player'], showarrow=False, name='Team',
            font=dict(size=11, 
                      color=teamColours[data.loc[row, 'Team']]['text']),
            bgcolor=teamColours[data.loc[row, 'Team']]['colour'], opacity=0.8, 
            borderpad=2.5)
fig.update_layout(showlegend=False, width=750, height=750, 
                  xaxis_title='Games Played',
                  yaxis_title='GSAx')
fig.update_yaxes(range=[(data["GSAx"].min() - 5), (data["GSAx"].max() + 5)])
fig.show()

In [300]:
fig = px.scatter(data, x='xGA', y='GA/100 xGA')
for row in data.index:
    fig.add_annotation(x=data.loc[row, 'xGA'], y=data.loc[row, 'GA/100 xGA'], 
            text=data.loc[row, 'Player'], showarrow=False, name='Team',
            font=dict(size=11, 
                      color=teamColours[data.loc[row, 'Team']]['text']),
            bgcolor=teamColours[data.loc[row, 'Team']]['colour'], opacity=0.8, 
            borderpad=2.5)
fig.update_layout(showlegend=False, width=750, height=750, 
                  xaxis_title='xGA Faced',
                  yaxis_title='Goals Against per 100 xGA')
fig.update_xaxes(range=[(data["xGA"].min() - 5), (data["xGA"].max() + 5)])
fig.update_yaxes(autorange="reversed")
fig.show()